In [ ]:
# Enable live visualization, automatic reload of modules upon cell execution.
%matplotlib widget
%load_ext autoreload
%autoreload 2

# Cahn-Hilliard model of mesenchymal clustering

In [ ]:
def free_energy(Lambda = 7):
    """
    Returns the energy derivatives. Uses SymPy for symbolical derivation.
    
    Parameters:
        Lambda: multiplier for the last term in F0 (see Boyer 2006 for discussion).
                May use 0 for partial spreading cases; for total spreading integer > 0.
    Returns:
        A pair of SymPy expressions for energy derivatives dg/dc1, dg/dc2
    """

    # Symbols for computing the derivatives of the energy F0. 
    # cx = concentrations; sxy = surface tensions; kx = see below; A = Lambda
    c1,c2,c3,s12,s13,s23,k1,k2,k3,A = sp.symbols('c1 c2 c3 s12 s13 s23 k1 k2 k3 A')

    # Surface tensions as the kappa parameters.
    s12 = (k1 + k2) / 2
    s13 = (k1 + k3) / 2
    s23 = (k2 + k3) / 2

    # Energy suitable for total spreading. See above & Boyer Eq. (38).
    F0 = s12*c1**2*c2**2 + s13*c1**2*c3**2 + s23*c2**2*c3**2 + c1*c2*c3*(k1*c1 + k2*c2 + k3*c3) + 3*A*c1**2*c2**2*c3**2

    # See Sigma_T between Eqs. (7) and (8) in Boyer 2006
    St = 3 / (1/k1 + 1/k2 + 1/k3)
    
    # See Boyer Eq. (8). Excluding epsilon.
    mu1 = 4*St * (1/k2*(sp.diff(F0,c1) - sp.diff(F0,c2)) + 1/k3*(sp.diff(F0,c1) - sp.diff(F0,c3)))
    mu1 = mu1.subs({A: str(Lambda), c3: 1-c1-c2})
    Fc1 = sp.collect(sp.simplify(mu1), c1) / 12    # TODO: don't divide by 12 here
    
    mu2 = 4*St * (1/k1*(sp.diff(F0,c2) - sp.diff(F0,c1)) + 1/k3*(sp.diff(F0,c2) - sp.diff(F0,c3)))
    mu2 = mu2.subs({A: str(Lambda), c3: 1-c1-c2})
    Fc2 = sp.collect(sp.simplify(mu2), c2) / 12    # TODO: don't divide by 12 here
   
    return Fc1, Fc2

## Numerical simulations

In [ ]:
import CH_solver as CH
import random
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# Model parameters. Note that these will be overwritten below as needed.
Ny_pdgfraHigh = 4      # thickness of middle (clustering) phase as nof. nodes
Ny_pdgfraLow = 13      # thickness of bottom phase as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration in the middle phase
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase
N       = [100, 30]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
T       = 0.001                 # total time
dt      = float(0.0001) / N[0]  # time step
M       = (0.1, 0.1, 0.1)       # phase mobilities
rng_seed = 2

### Figs. 5B, S6N: Thin layer break-up test

Simulates the formation of clusters arising from the break-up of a thin noise layer layer (e.g., Pdgfra+ cells) sandwiched between two thick layers (Pdgfra- mesenchyme and epithelium).

In [ ]:
# 
# Create the phases figure.
# 
nrows = 4   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.3

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)

plt.show()

In [ ]:
import CH_solver as CH

# Model parameters
Ny_pdgfraHigh = 4      # thickness of middle (clustering) phase as nof. nodes
Ny_pdgfraLow = 13      # thickness of bottom phase as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration in the middle phase
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase
N       = [100, 30]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
T       = 0.001                 # total time
dt      = float(0.0001) / N[0]  # time step
M       = (0.1, 0.1, 0.1)       # phase mobilities. TODO: check that these are handled correctly


def IC(self, values, x):    
    """
    Sets the initial phase/tissue concentrations.
   
    Assumes the following global variables:
    Ny_pdgfraHigh - thickness of middle (clustering) phase as nof. nodes
    Ny_pdgfraLow  - thickness of bottom phase as nof. nodes
    p_pdgfraHigh  - initial Pdgfra^High concentration in the middle phase 
    p_pdgfraLow   - initial Pdgfra^High concentration in the bottom phase

    Parameters:
        values: array of 4 floats corresponding to phase 1 (epithelial) 
                concentration, phase 2 (Pdgfra^Low) concentration and 
                mu1, mu2, respectively.
        x: array of 2 floats indicating the position in the domain, scaled
           to range [0,1] for the longest axis.
    """
    yc = x[1]*N[0]
    # Epithelium
    if (yc > Ny_pdgfraLow + Ny_pdgfraHigh):
        values[0] = 1.0
        values[1] = 0.0 
    # Pdgfra^High
    elif (yc > Ny_pdgfraLow):                 
        values[0] = 0.0
        p = (random.random() - 0.5) / 2
        values[1] = (1.0 - p_pdgfraHigh) + p
        values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)
    # Pdgfra^Low
    else:
        values[0] = 0.0
        values[1] = 1.0
        # TODO: fix the weird logic of setting the noisy concentrations
        if (p_pdgfraLow > 0.0):
            p = (random.random() - 0.5) / 2
            values[1] = (1.0 - p_pdgfraLow) + p
            values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)

    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2

    
# ** WT **
s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, T, kappa, M, 
                    output="breakup_test/WT", 
                    title=("\nWT -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** Low E-G adhesion **
s12 = 0.75    # EM
s13 = 1.25    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 5*dt, T, kappa, M, 
                    output="breakup_test/LowEG",
                    title=("\nLow E-G adhesion -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** Low G-G cohesion **
s12 = 0.75    # EM
s13 = 0.25    # EG
s23 = 0.50    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 5*dt, T, kappa, M, 
                    output="breakup_test/LowGG",
                    title=("\nLow G-G cohesion -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** Low G-M adhesion (NOTE: total spreading) **
s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 2.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[3], N, eps, dt, T, kappa, M, 
                    output="breakup_test/LowGM",
                    title=("\nLow G-M adhesion -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))


In [ ]:
plt.savefig('Fig5B-S6N.eps', bbox_inches='tight')
plt.savefig('Fig5B-S6N.png', bbox_inches='tight') #, pad_inches=0)

### Fig. 5C: Pattern scaling

Testing the effect of initial green layer density to cluster size and density.

In [ ]:
nrows = 3   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.3
import CH_solver as CH

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
import CH_solver as CH

rng_seed = 1
noise_amplitude = 0.5   # magnitude of noise around p_pdgfraHigh


def randomInitConcentration(p_pdgfraHigh, Ny_pdgfraHigh, Ny_pdgfraLow):
    """
    Generates a random sequence of concentration values for clustering 
    phase concentrations, centered at p_pdgfraHigh and such that the mean
    of the sequence is exactly p_pdgfraHigh.
    """
    np.random.seed(rng_seed)
    N1 = [N[0]+1, N[1]+1]   # padding +1 nodes since fenics interpolator may access those
    cArray = np.zeros(N1).transpose()
    # Sequence of random numbers around p_pdgfraHigh
    cRand = p_pdgfraHigh + noise_amplitude * (np.random.rand(Ny_pdgfraHigh*N1[0], 1) - 0.5)
    cRand = np.clip(cRand, a_min=0.0, a_max=1.0)
    # Scale the sequence so that the mean value equals (up to rounding) to p_pdgfraHigh
    cRand = p_pdgfraHigh * cRand/sum(cRand) * len(cRand)
    # Assign the values to the position corresponding to pdgfraHigh nodes.
    i0 = Ny_pdgfraLow
    i1 = i0 + Ny_pdgfraHigh
    cArray[i0:i1, :] = cRand.reshape(Ny_pdgfraHigh, N1[0])
    
    return cArray


def IC(self, values, x):
    # Gives the vertical (yc) and horizontal (xc) array coordinate index 
    # such that 0 = bottom of the pdgfraLow mesenchyme. E.g., in the case 
    # of [100, 30] nodes, x[1] is in range [0.0, 0.3] => yc is in [0, 30].
    yc = x[1]*N[0] 
    xc = x[0]*N[0]
    
    # Epithelium
    if (yc > Ny_pdgfraLow + Ny_pdgfraHigh):
        values[0] = 1.0
        values[1] = 0.0
    # Pdgfra^High
    elif (yc > Ny_pdgfraLow):   
        values[0] = 0.0
        values[1] = 1.0 - cArray[int(yc),int(xc)]
        # values[0] = 0.0
        # p = (random.random() - 0.5) / 2
        # values[1] = (1.0 - p_pdgfraHigh) + p
        # values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)
    # Pdgfra^Low
    else:
        values[0] = 0.0
        values[1] = 1.0
        if (p_pdgfraLow > 0.0):
            p = (random.random() - 0.5) / 2
            values[1] = (1.0 - p_pdgfraLow) + p
            values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)

    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2


# ** Thin **
s12 = 0.75    # CX
s13 = 0.75    # CG
s23 = 1.00    # RG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
Ny_pdgfraHigh = 4
Ny_pdgfraLow = 13
p_pdgfraHigh = 0.55
p_pdgfraLow = 0.0
cArray = randomInitConcentration(p_pdgfraHigh, Ny_pdgfraHigh, Ny_pdgfraLow)
matTotal = Ny_pdgfraHigh * p_pdgfraHigh
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, 0.5*T, kappa, M, rng_seed=rng_seed,
                    output="initial_material/low", 
                    title=("\nLow (pdgfraHigh=%g) -- kE=%g, kM=%g, kG=%g" %(matTotal, kappa[0], kappa[1], kappa[2])))

# ** WT **
s12 = 0.75    # CX
s13 = 0.75    # CG
s23 = 1.00    # RG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
Ny_pdgfraHigh = 5
Ny_pdgfraLow = 13
p_pdgfraHigh = 0.52
p_pdgfraLow = 0.0
cArray = randomInitConcentration(p_pdgfraHigh, Ny_pdgfraHigh, Ny_pdgfraLow)
matTotal = Ny_pdgfraHigh * p_pdgfraHigh 
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    output="initial_material/med_WT", 
                    title=("\nMed/WT (pdgfraHigh=%g) -- kE=%g, kM=%g, kG=%g" %(matTotal, kappa[0], kappa[1], kappa[2])))

# ** Thick **
s12 = 0.75    # CX
s13 = 0.75    # CG
s23 = 1.00    # RG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
Ny_pdgfraHigh = 6
Ny_pdgfraLow = 13
p_pdgfraHigh = 0.5
p_pdgfraLow = 0.0
cArray = randomInitConcentration(p_pdgfraHigh, Ny_pdgfraHigh, Ny_pdgfraLow)
matTotal = Ny_pdgfraHigh * p_pdgfraHigh
dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    output="initial_material/high",
                    title=("\nHigh (pdgfraHigh=%g) -- kE=%g, kM=%g, kG=%g" %(matTotal, kappa[0], kappa[1], kappa[2])))        

In [ ]:
# plt.savefig('initial_material/scaling.eps', bbox_inches='tight')
plt.savefig('initial_material/seed1.png', bbox_inches='tight')

### Fig. 5G: Cohesion test

Controlling green cohesion in a stepwise manner from high-to-low cohesion scenarios.

In [ ]:
# Create the phases figure.
# fig, axes = plt.subplots(nrows=7, ncols=1, constrained_layout=True)
# fig.set_figheight(1.2*len(axes))
# [ax.xaxis.set_major_locator(plt.NullLocator()) for ax in axes]
# [ax.yaxis.set_major_locator(plt.NullLocator()) for ax in axes]

nrows = 6   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.3

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)

plt.show()

In [ ]:
Ny_pdgfraHigh = 4      # thickness of Pdgfra^High (clustering phase) as nof. nodes
Ny_pdgfraLow = 13      # thickness of Pdgfra^Low (bottom phase) as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration vs. PdgfraLow
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase
rng_seed = 2
cArray = randomInitConcentration(p_pdgfraHigh, Ny_pdgfraHigh, Ny_pdgfraLow)

# ** Low G-G cohesion **
s12 = 0.75    # EM
s13 = 0.25    # EG
s23 = 0.50    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    title=("\nLow G-G cohesion -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

s12 = 0.75    # EM
s13 = 0.45    # EG
s23 = 0.70    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                   title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

s12 = 0.75    # EM
s13 = 0.55    # EG
s23 = 0.80    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

s12 = 0.75    # EM
s13 = 0.65    # EG
s23 = 0.90    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[3], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** WT **
s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[4], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    title=("\nWT -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** High G-G cohesion **
s12 = 0.75    # EM
s13 = 1.05    # EG
s23 = 1.30    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[5], N, eps, 5*dt, T, kappa, M, rng_seed=rng_seed,
                    title=("\nHigh G-G cohesion -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))


In [ ]:
plt.savefig('green_cohesion.eps', bbox_inches='tight')
plt.savefig('green_cohesion.png', bbox_inches='tight')

### Fig. 5F: Deep clusters

In [ ]:
# Create the phases figure.
nrows = 2   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.7

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
import CH_solver as CH

N       = [100, 40]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
T       = 0.001                 # total time
dt      = float(0.0001) / N[0]  # time step
M       = (0.1, 0.1, 0.1)       # phase mobilities. TODO: check that these are handled correctly

rng_seed = 2

def IC(self, values, x):
    """
    Parameters:
        values: array of 4 floats for phase1, phase2 concentrations and mu1, mu2 at position x.
        x: 2D position array of 2 floats in [0.0, 1.0] x [0.0, N[0]/N[1]].
    """    
    yc = x[1]*N[0]    # scales y coordinates to range [0, N[1]]
    # Epithelium
    if (yc > Ny_pdgfraLow + Ny_pdgfraHigh):
        values[0] = 1.0
        values[1] = 0.0 
    # Pdgfra^High
    elif (yc > Ny_pdgfraLow):                 
        values[0] = 0.0
        p = (random.random() - 0.5) / 2
        values[1] = (1.0 - p_pdgfraHigh) + p
        values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)
    # Pdgfra^Low-High mixture
    elif (yc > -1):   # designating bottom 4 nodes as pure Pdgfra^Low phase
        values[0] = 0.0
        values[1] = 1.0
        if (p_pdgfraLow > 0.0):
            p = (random.random() - 0.5) / 2
            values[1] = (1.0 - p_pdgfraLow) + p
            values[1] = np.clip(values[1], a_min=0.0, a_max=1.0)
    else:
        values[0] = 0.0
        values[1] = 1.0            
            
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2
    
# ** No thick interface layer **
s12 = 0.75
s13 = 0.75
s23 = 1.00
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
Ny_pdgfraHigh = 2
Ny_pdgfraLow = 24
p_pdgfraHigh = 0.30   
p_pdgfraLow = 0.30
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, 0.5*T, kappa, M,
                    output="deep_clustering/no-thick-interface",
                    title=("\nThick interface -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** Thick interface layer **
s12 = 0.75    # CX
s13 = 0.75    # CG
s23 = 1.00    # RG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
Ny_pdgfraHigh = 2
Ny_pdgfraLow = 24
p_pdgfraHigh = 0.5   
p_pdgfraLow = 0.30
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 5*dt, 0.5*T, kappa, M,
                    output="deep_clustering/thick-interface",
                    title=("\nThick interface -- kE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))


In [ ]:
plt.savefig('Fig5D_deep_clustering.eps', bbox_inches='tight')
plt.savefig('Fig5D_deep_clustering.png', bbox_inches='tight')

### Fig. 5K: Coarsening

In [ ]:
# Create the phases figure.
nrows = 1   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.2

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)
axes = [axes]

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
N       = [100, 30]             # domain nodes (x,y)

Ny_pdgfraHigh = 4      # thickness of middle (clustering) phase as nof. nodes
Ny_pdgfraLow = 13      # thickness of bottom phase as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration in the middle phase
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase

# ** WT **
s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, 10*T, kappa, M, output="Fig5E_coarsening")

### Simulation X: Aggregates

In [ ]:
import CH_solver as CH
import random
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# Model parameters
Ny_pdgfraHigh = 4      # thickness of middle (clustering) phase as nof. nodes
Ny_pdgfraLow = 13      # thickness of bottom phase as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration in the middle phase
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase
# N       = [70, 30]             # domain nodes (x,y)
N       = [210, 90]
eps     = 2 / (N[0]-1)          # interface thickness
T       = 0.001                 # total time
dt      = float(0.0001) / N[0]  # time step
M       = (0.1, 0.1, 0.1)       # phase mobilities. TODO: check that these are handled correctly


def IC(self, values, x):
    """
    Sets the initial phase/tissue concentrations.
   
    Assumes the following global variables:
    Ny_pdgfraHigh - thickness of middle (clustering) phase as nof. nodes
    Ny_pdgfraLow  - thickness of bottom phase as nof. nodes
    p_pdgfraHigh  - initial Pdgfra^High concentration in the middle phase 
    p_pdgfraLow   - initial Pdgfra^High concentration in the bottom phase

    Parameters:
        values: array of 4 floats corresponding to phase 1 (epithelial) 
                concentration, phase 2 (Pdgfra^Low) concentration and 
                mu1, mu2, respectively.
        x: array of 2 floats indicating the position in the domain, scaled
           to range [0,1] for the longest axis.
    """
    
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    # Epithelium
    '''
        pnorm = np.sqrt(xc*xc + yc*yc)
    values[0] = 0.5*(1 + np.tanh(2/eps * (pnorm-r)))
    values[1] = 0.5*(1 - np.tanh(2/eps * (-pnorm+r)))
    if (yc < pnorm-r):
        values[0] = 0.5*(1 + np.tanh(2/eps * yc))   # c1
    if (yc > -pnorm+r):
        values[1] = 0.5*(1 - np.tanh(2/eps * yc))   # c2
    '''        
    if (xc-0.06)**2 + yc**2 < 0.005 or (xc+0.06)**2 + yc**2 < 0.005: 
        values[0] = 0.0
        values[1] = 0.0 
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2


In [ ]:
nrows = 9   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.3

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]


print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
#
# First simulating 2-phase aggragation at 3 different adhesion levels.
#

s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)

#
# Two-phase: one cell type + media
#

def IC(self, values, x):
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    if (xc-0.13)**2 + yc**2 < 0.016 or (xc+0.13)**2 + yc**2 < 0.016: 
    # if (xc-0.11)**2 + yc**2 < 0.02 or (xc+0.11)**2 + yc**2 < 0.02: 
        values[0] = 0.0
        values[1] = 1.0 
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2
    

# ** High G cohesion **
# s12 = 0.75    # EM
# s13 = 1.75    # EG
# s23 = 1.75    # MG
kappa = (s12+s13-s23, 4*(s12+s23-s13), (s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 2.0*dt, 8*T, kappa, M, 
                    output="aggregate_highGG_4x",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** WT **
# s12 = 0.75    # EM
# s13 = 0.75    # EG
# s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 2.0*dt, 8*T, kappa, M, 
                    output="aggregate_WT",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# Low G cohesion:
# s12 = 0.75    # EM
# s13 = 0.50    # EG
# s23 = 0.50    # MG
# kappa = (s12+s13-s23, 0.125*(s12+s23-s13), s13+s23-s12)
# dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 10.0*dt, 2*T, kappa, M, 
#                     output="aggregate_lowGG",
#                     title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))


#
# Three-phase: two cell types + media
#

def IC(self, values, x):
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    if (xc-0.13)**2 + yc**2 < 0.016 or (xc+0.13)**2 + yc**2 < 0.016: 
    # if (xc-0.06)**2 + yc**2 < 0.005 or (xc+0.06)**2 + yc**2 < 0.005: 
        values[0] = 0.0
        values[1] = 1.0 - random.random()
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2
'''
# ** High G cohesion **
# s12 = 0.75    # EM
# s13 = 1.75    # EG
# s23 = 1.75    # MG
kappa = (s12+s13-s23, (s12+s23-s13), 8.0*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[3], N, eps, 10.0*dt, 2*T, kappa, M, 
                    output="aggregate_highGG_mix",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** WT **
# s12 = 0.75    # EM
# s13 = 0.75    # EG
# s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[4], N, eps, 10.0*dt, 2*T, kappa, M, 
                    output="aggregate_WT_mix",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# Low G cohesion:
# s12 = 0.75    # EM
# s13 = 0.50    # EG
# s23 = 0.50    # MG
kappa = (s12+s13-s23, (s12+s23-s13), 0.125*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[5], N, eps, 2.0*dt, 2*T, kappa, M, 
                    output="aggregate_lowGG_mix",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))
'''

#
# Three-phase: two cell types + media, lower G concentration.
#

def IC(self, values, x):
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    if (xc-0.13)**2 + yc**2 < 0.016 or (xc+0.13)**2 + yc**2 < 0.016: 
    # if (xc-0.06)**2 + yc**2 < 0.005 or (xc+0.06)**2 + yc**2 < 0.005: 
        values[0] = 0.0
        values[1] = 1.0 - 0.8*random.random()
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2

'''
# ** High G cohesion **
# s12 = 0.75    # EM
# s13 = 1.75    # EG
# s23 = 1.75    # MG
kappa = (s12+s13-s23, (s12+s23-s13), 8.0*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[6], N, eps, 5.0*dt, 2*T, kappa, M, 
                    output="aggregate_highGG_mix_0.5xcG",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** WT **
# s12 = 0.75    # EM
# s13 = 0.75    # EG
# s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[7], N, eps, 5.0*dt, 2*T, kappa, M, 
                    output="aggregate_WT_mix_0.5xcG",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))
'''
'''
# Low G cohesion:
# s12 = 0.75    # EM
# s13 = 0.50    # EG
# s23 = 0.50    # MG
kappa = (s12+s13-s23, 8*(s12+s23-s13), (s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[8], N, eps, 5.0*dt, 2*T, kappa, M, 
                    output="aggregate_lowGG_mix_0.5xcG",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))
'''


In [ ]:
plt.savefig('aggregation.eps', bbox_inches='tight')
plt.savefig('aggregation.png', bbox_inches='tight')

In [ ]:
nrows = 12   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.3

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]


print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
#
# First simulating 2-phase aggregation at 3 different adhesion levels.
#

s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)

#
# Two-phase: one cell type + media
#

def IC(self, values, x):
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    if xc**2 + yc**2 < 0.025: 
        values[0] = 0.0
        values[1] = 1.0 - random.random()/2
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2

'''
# ** High M cohesion **
kappa = (s12+s13-s23, 8.0*(s12+s23-s13), s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_1",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** High G cohesion **
kappa = (s12+s13-s23, (s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_2",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

# ** WT **
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_3",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, 2*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[3], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_4",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, 4*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[4], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_5",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))
'''

kappa = (s12+s13-s23, 8*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[5], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_6",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

def IC(self, values, x):
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    if xc**2 + yc**2 < 0.025: 
        values[0] = 0.0
        values[1] = 1.0 - 2*random.random()/3 
    elif yc > 0.0:
        values[0] = 1.0
        values[1] = 0.0 
    else:
        values[0] = 1.0
        values[1] = 0.0 
        
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2

kappa = (s12+s13-s23, 8.0*(s12+s23-s13), s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[6], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_7",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, (s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[7], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_8",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[8], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_9",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, 2*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[9], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_10",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, 4*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[10], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_11",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))

kappa = (s12+s13-s23, 8*(s12+s23-s13), 8*(s13+s23-s12))
dnorm = CH.simulate(IC, free_energy, fig, axes[11], N, eps, 2.0*dt, 4*T, kappa, M, 
                    output="aggregate_single_12",
                    title=("\nkE=%g, kM=%g, kG=%g" %(kappa[0], kappa[1], kappa[2])))


In [ ]:
plt.savefig('aggregation.eps', bbox_inches='tight')
plt.savefig('aggregation.png', bbox_inches='tight')

### Valiation 1: Lens test 

Performs the classic finite boundary "lens test". See Boyanova 2014, Section 5 for more details. The lens test is useful for validation purposes, as the contact angle of the lens obtained from the simulation should correspond to the theoretical prediction given by...

In [ ]:
# Create the phases figure.
nrows = 1   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.4

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)
axes = [axes]

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=1.0)
plt.show()

In [ ]:
N       = [100, 50]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
dt      = float(0.0001) / N[0]  # time step
T       = 0.001                 # total time
M       = (0.1, 0.1, 0.1)       # phase mobilities

# Phase initial conditions.
def IC(self, values, x):
    r = 0.15    # lens radius
    xc = x[0] - 0.5
    yc = x[1] - 0.5 * (N[1]/N[0])
    pnorm = np.sqrt(xc*xc + yc*yc)
    values[0] = 0.5*(1 + np.tanh(2/eps * (pnorm-r)))
    values[1] = 0.5*(1 - np.tanh(2/eps * (-pnorm+r)))
    if (yc < pnorm-r):
        values[0] = 0.5*(1 + np.tanh(2/eps * yc))   # c1
    if (yc > -pnorm+r):
        values[1] = 0.5*(1 - np.tanh(2/eps * yc))   # c2
    values[2] = 0.0         # mu1
    values[3] = 0.0         # mu2

# Surface energies sij between phases i,j
s12 = 0.75    # CX
s13 = 0.75    # CG
s23 = 1.00    # RG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
    
# Run the simulation loop, update figure.
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, 50*T, kappa, M, output="lens_test")

### Demo 1: Phase separation in a noisy field

Basic Cahn-Hilliard demonstration; phase-separation of three phases starting from a noisy mixed state.

In [ ]:
import CH_solver as CH
import random
import matplotlib.pyplot as plt

#
# Model parameters
#
N       = [100, 50]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
dt      = float(0.0002) / N[0]  # time step
T       = 15*dt # 0.000512              # total time

# Surface energies sij between phases i,j
s12 = 1.0
s13 = 0.8
s23 = 1.4
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)

M = (0.1, 0.1, 0.1)

# Phase initial conditions.
def IC(self, values, x):
    values[0] = 0.0     # c1
    values[1] = 0.0     # c2
    values[2] = 0.0     # mu1
    values[3] = 0.0     # mu2
    p = random.random()
    if (p < 1/3):
        values[0] = 1.0
    elif (p < 2/3):
        values[1] = 1.0
    else:
        pass

# Create the phases figure.
fig = plt.figure()
plt.show()

In [ ]:
# Run the simulation loop, update figure.
dnorm = CH.simulate(IC, free_energy, fig, N, eps, dt, T, kappa, M)

### Supplement 1: Spreading coefficients and clustering


In [ ]:
Ny_pdgfraHigh = 4      # thickness of middle (clustering) phase as nof. nodes
Ny_pdgfraLow = 13      # thickness of bottom phase as nof. nodes
p_pdgfraHigh = 0.65    # initial Pdgfra^High concentration in the middle phase
p_pdgfraLow = 0.0      # initial Pdgfra^High concentration in the bottom phase
N       = [100, 30]             # domain nodes (x,y)
eps     = 2 / (N[0]-1)          # interface thickness
T       = 0.001                 # total time
dt      = float(0.0001) / N[0]  # time step
M       = (0.1, 0.1, 0.1)       # phase mobilities. TODO: check that these are handled correctly

In [ ]:
# 
# Create the phases figure.
# 
nrows = 3   # number of simulations to show
width = 6   # figure width in inches
height = width*nrows * N[1]/N[0] * 1.2

fig, axes = plt.subplots(figsize=(width,height), nrows=nrows, ncols=1) #, constrained_layout=True)

[ax.set_aspect(N[1]/N[0]) for ax in axes]
[ax.axis('off') for ax in axes]
print("Fig. size", fig.get_size_inches(), ", dpi", fig.dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=0.95)
plt.show()

In [ ]:
# Surface energies sij between phases i,j

#
# 2022-02-13 integrin figure candidates (1=(E)pithelium, 2=(M)esenchyme, 3=(G)fp high)
#
    
# ** WT **
s12 = 0.75    # EM
s13 = 0.75    # EG
s23 = 1.00    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[0], N, eps, 5*dt, T, kappa, M, output="Fig5S_WT")

# ** Neutral **
s12 = 5/6    # EM
s13 = 5/6    # EG
s23 = 5/6    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[1], N, eps, 5*dt, T, kappa, M, output="Fig5S_LowEG")

# ** Low G-X adhesion (circular clusters) **
s12 = 0.75    # EM
s13 = 2.5    # EG
s23 = 2.5    # MG
kappa = (s12+s13-s23, s12+s23-s13, s13+s23-s12)
dnorm = CH.simulate(IC, free_energy, fig, axes[2], N, eps, 5*dt, T, kappa, M, output="Fig5S_lowG")


In [ ]:
plt.savefig('Fig5S_adhesion.eps', bbox_inches='tight')
plt.savefig('Fig5S_adhesion.png', bbox_inches='tight') #, pad_inches=0)